In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg19 import preprocess_input
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
df = pd.read_csv('combined.csv')

In [ ]:
label_array = df[' Label'].unique()
count = 0
for i in label_array:
  df.loc[(df[' Label']== i),' Label']= count
  print (i , count )
  count += 1

BENIGN 0
DDoS 1
PortScan 2
Bot 3
Infiltration 4
Web Attack � Brute Force 5
Web Attack � XSS 6
Web Attack � Sql Injection 7
FTP-Patator 8
SSH-Patator 9
DoS slowloris 10
DoS Slowhttptest 11
DoS Hulk 12
DoS GoldenEye 13
Heartbleed 14


In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
df = clean_dataset(df)

In [ ]:
dataset = df.values

In [ ]:
X = dataset[:,0:78]

In [ ]:
Y =dataset[:,78]

In [ ]:
Y=Y.astype(int)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train,X_val_test,Y_train,Y_val_test = train_test_split(X,Y,test_size = 0.3)

In [ ]:
X_val,X_test,Y_val,Y_test = train_test_split(X_val_test,Y_val_test,test_size = 0.5)

In [ ]:
print(X_train.shape)

(1979513, 78)


In [ ]:
X_train = np.array([x.reshape(78,1) for x in X_train])
X_test = np.array([x.reshape(78,1) for x in X_test])
X_val = np.array([x.reshape(78,1) for x in X_val])

In [ ]:
Y_train = np.array([x.reshape(1) for x in Y_train])
Y_test = np.array([x.reshape(1) for x in Y_test])
Y_val = np.array([x.reshape(1) for x in Y_val])

In [ ]:
X_train.shape

(1979513, 78, 1)

In [ ]:
Y_train.shape

(1979513, 1)

In [42]:
model = models.Sequential()
model.add(layers.Conv1D(32,5,activation='sigmoid',input_shape =(78,1)))
model.add(layers.Conv1D(32,5,activation='sigmoid'))
model.add(layers.MaxPool1D(2,1))
model.add(layers.Conv1D(32,5,activation = 'sigmoid'))
model.add(layers.MaxPool1D(2,1))
model.add(layers.BatchNormalization(batch_size = 32))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='softmax'))
model.add(layers.Dense(15))

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 74, 32)            192       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 70, 32)            5152      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 69, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 65, 32)            5152      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 64, 32)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32)            128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 32)           

In [44]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=10, 
                    validation_data=(X_val, Y_val))

Epoch 1/10
61860/61860 [==============================] - 295s 5ms/step - loss: 0.4326 - accuracy: 0.9043 - val_loss: 0.0655 - val_accuracy: 0.9727
Epoch 2/10
61860/61860 [==============================] - 292s 5ms/step - loss: 0.0659 - accuracy: 0.9730 - val_loss: 0.0504 - val_accuracy: 0.9813
Epoch 3/10
61860/61860 [==============================] - 300s 5ms/step - loss: 0.0538 - accuracy: 0.9773 - val_loss: 0.0472 - val_accuracy: 0.9803
Epoch 4/10
61860/61860 [==============================] - 304s 5ms/step - loss: 0.0497 - accuracy: 0.9791 - val_loss: 0.0412 - val_accuracy: 0.9850
Epoch 5/10
61860/61860 [==============================] - 299s 5ms/step - loss: 0.0470 - accuracy: 0.9806 - val_loss: 0.0397 - val_accuracy: 0.9824
Epoch 6/10
61860/61860 [==============================] - 299s 5ms/step - loss: 0.0454 - accuracy: 0.9812 - val_loss: 0.0404 - val_accuracy: 0.9847
Epoch 7/10
61860/61860 [==============================] - 299s 5ms/step - loss: 0.0437 - accuracy: 0.9822 - val_

In [45]:
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=2)

13256/13256 - 25s - loss: 0.0373 - accuracy: 0.9857


In [46]:
predictions = model.predict(X_test)

In [49]:
predictions[1577]

array([ -1.6217293, -13.742586 ,  -8.458064 , -35.684155 , -22.323708 ,
        -3.1333838,  -3.740532 , -10.248486 , -21.918392 , -20.890314 ,
       -19.317839 , -36.04591  , -13.049951 ,   3.7284348, -17.454838 ],
      dtype=float32)

In [51]:
X_test[1577]

array([[-4.36762359e-01],
       [-9.41247873e-02],
       [-3.15840755e-03],
       [-5.41536574e-03],
       [-2.00879534e-02],
       [-2.00819607e-03],
       [ 1.96783301e-01],
       [-3.10317193e-01],
       [-4.51146036e-02],
       [ 2.23699812e-01],
       [ 3.29519974e+00],
       [-5.96523229e-01],
       [ 3.33640163e+00],
       [ 3.58981302e+00],
       [-5.74664747e-02],
       [-2.78494161e-01],
       [-5.37328305e-02],
       [-6.73967351e-02],
       [-1.04810900e-01],
       [-5.50666494e-02],
       [-2.34216027e-01],
       [-1.58210030e-01],
       [-5.84703706e-02],
       [-9.87203399e-02],
       [-1.19004351e-01],
       [ 5.89972762e-02],
       [ 1.23747181e-01],
       [ 3.07968123e-01],
       [ 1.12854656e-01],
       [-1.16452213e-01],
       [-2.20574856e-01],
       [ 0.00000000e+00],
       [-1.05547824e-02],
       [ 0.00000000e+00],
       [ 1.24650535e-03],
       [ 1.68181793e-03],
       [-2.58140178e-01],
       [-1.83436862e-01],
       [-6.5